In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from torch.autograd import Variable

PyTorch Version:  1.1.0
Torchvision Version:  0.3.0


## GPU & mount Google Drive


In [45]:
from google.colab import drive
drive.mount('/content/gdrive')

# with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
#   f.write('Hello Google Drive!')
# !cat /content/gdrive/My\ Drive/foo.txt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


How to use gpu in Colab: https://medium.com/@nrezaeis/pytorch-in-google-colab-640e5d166f13

In [3]:
#check the version of Cuda and python
!nvcc --version
!python --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
Python 3.6.8


command: https://pytorch.org/

In [5]:
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [6]:
torch.cuda.get_device_name(0)

'Tesla K80'

# Global Variables

In [0]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

model_name = 'resnet'

# Number of classes in the dataset
num_classes = 10

# Batch size for training (change depending on how much memory you have)
batch_size = 32

# Number of epochs to train for
num_epochs = 20

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

input_size = 224

num_workers = 4

device = torch.device("cuda")
use_gpu=True

# Helper Functions

## Model training and validation code
Use of fstring to concatenate strings: https://realpython.com/python-f-strings/

In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
#     model_save_name = f"{num_epochs}.pt"
#     PATH= f"gdrive/My Drive/201906AI_FinalProject/checkpoints_resnet18/{model_save_name}"
#     torch.save({'epoch': 0, 'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'loss': 0}, PATH)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode"
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)                  
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()
    
#     model_save_name = f"{epoch}.pt"
#     PATH= f"gdrive/My Drive/201906AI_FinalProject/checkpoints_resnet18/{model_save_name}"
#     torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'loss': loss}, PATH)
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

## Set Model Parameters' requires_grad attribute

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Initialize and reshape the network

In [10]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    model_ft = models.resnet18(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

# Load Data

In [11]:
data_transforms = {
    'train': transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(224), 
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

train_dir = 'gdrive/My Drive/201906AI_FinalProject/data_all/images/train'
val_dir = 'gdrive/My Drive/201906AI_FinalProject/data_all/images/val'
test_dir =  'gdrive/My Drive/201906AI_FinalProject/data_all/images/test'

train_dset =  datasets.ImageFolder(train_dir, transform=data_transforms['train'])
train_loader = torch.utils.data.DataLoader(train_dset,
                     batch_size=batch_size,
                     num_workers=num_workers,
                     shuffle=True)

val_dset =  datasets.ImageFolder(val_dir, transform=data_transforms['val'])
val_loader =torch.utils.data.DataLoader(val_dset,
                  batch_size=batch_size,
                  num_workers=num_workers)

test_dset =  datasets.ImageFolder(test_dir, transform=data_transforms['test'])
test_loader = torch.utils.data.DataLoader(test_dset,
                     batch_size=batch_size,
                     num_workers=num_workers,
                     shuffle=True)

image_datasets = {'train': train_dset, 'val':val_dset, 'test': test_dset}
dataloaders_dict = {'train': train_loader, 'val':val_loader, 'test':test_loader}


dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val', 'test']}

for x in ['train', 'val', 'test']:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets['test'].classes
print(image_datasets['test'].classes)

Loaded 12454 images under train
Loaded 4570 images under val
Loaded 489 images under test
Classes: 
['BernardPicart', 'GeorgeHendrikBreitner', 'IsaacIsraels', 'JanLuyken', 'JohannesTavenraat', 'MariusBauer', 'ReinierVinkeles', 'RembrandtHarmenszvanRijn', 'SimonFokke', 'WillemWitsen']


# Opitimizer

In [12]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)


# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

# Run Training an validation step

In [0]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [0]:
# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))#9:55

Epoch 0/19
----------
train Loss: 1.0593 Acc: 0.6337
val Loss: 0.7578 Acc: 0.7322

Epoch 1/19
----------
train Loss: 0.6574 Acc: 0.7691
val Loss: 0.7354 Acc: 0.7348

Epoch 2/19
----------
train Loss: 0.5428 Acc: 0.8152
val Loss: 0.5584 Acc: 0.8007

Epoch 3/19
----------
train Loss: 0.4829 Acc: 0.8275
val Loss: 0.9409 Acc: 0.6768

Epoch 4/19
----------
train Loss: 0.4281 Acc: 0.8502
val Loss: 0.5180 Acc: 0.8241

Epoch 5/19
----------
train Loss: 0.3966 Acc: 0.8594
val Loss: 0.4125 Acc: 0.8626

Epoch 6/19
----------
train Loss: 0.3651 Acc: 0.8749
val Loss: 0.5112 Acc: 0.8387

Epoch 7/19
----------
train Loss: 0.3414 Acc: 0.8813
val Loss: 0.5763 Acc: 0.8162

Epoch 8/19
----------
train Loss: 0.3187 Acc: 0.8895
val Loss: 0.5103 Acc: 0.8335

Epoch 9/19
----------
train Loss: 0.2993 Acc: 0.8951
val Loss: 0.6369 Acc: 0.8026

Epoch 10/19
----------
train Loss: 0.2848 Acc: 0.9002
val Loss: 0.4871 Acc: 0.8403

Epoch 11/19
----------


# Test 
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [0]:
ckpath = "gdrive/My Drive/201906AI_FinalProject/checkpoints_resnet18/19.pt"

In [32]:
#     model_save_name = f"{epoch}.pt"
#     PATH= f"gdrive/My Drive/201906AI_FinalProject/checkpoints_resnet18/{model_save_name}"
#     torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'loss': loss}, PATH)

print('===> Try resume from checkpoint')

checkpoint = torch.load(ckpath)
model_ft.load_state_dict(checkpoint['model_state_dict'])
optimizer_ft.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print('===> Load last checkpoint data')



===> Try resume from checkpoint
===> Load last checkpoint data


In [33]:
print(loss)

tensor(1.7806, device='cuda:0')


In [36]:
# 设置为评估模式
model_ft.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [37]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += torch.sum(predicted == labels)

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
    


Test Accuracy of the model on the test images: 93 %


In [53]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        model_ft = model_ft.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(9):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    if class_total[i]==0:
      continue
    print('Accuracy of %5s : %2d %%' % (image_datasets['test'].classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of BernardPicart : 87 %
Accuracy of GeorgeHendrikBreitner :  0 %
Accuracy of JanLuyken : 93 %
Accuracy of ReinierVinkeles : 62 %
Accuracy of RembrandtHarmenszvanRijn : 100 %
Accuracy of SimonFokke : 93 %


# 测试风格迁移后的图片

In [57]:
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = model_ft(images)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += torch.sum(predicted == labels)

#     print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))
    
imagepath = "gdrive/My Drive/201906AI_FinalProject/StyleTransfer/results"
with torch.no_grad():
  test1_dset = datasets.ImageFolder(imagepath, transform=data_transforms['test'])
  test1_loader = torch.utils.data.DataLoader(test1_dset,
                     batch_size=batch_size,
                     num_workers=num_workers,
                     shuffle=True)
  for images, labels in test1_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs, 1)
        print("Predicted results:", predicted)
 

Predicted results: tensor([5, 5, 1], device='cuda:0')
